In [1]:
import pyspark
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.getOrCreate()

In [2]:
from sample_agg_blb import SampleAggregate

In [3]:
from pyspark.sql.types import FloatType, BooleanType

filepath = "../../../dp-test-datasets/data/PUMS_california_demographics/data.csv"
pums = spark.read.load(filepath, format="csv", sep=",",inferSchema="true", header="true")

pums = pums.withColumnRenamed("_c0", "PersonID")

pums = pums.withColumn("income", col("income").cast(FloatType()))
pums = pums.withColumn("latino", col("latino").cast(BooleanType()))
pums = pums.withColumn("black", col("black").cast(BooleanType()))
pums = pums.withColumn("asian", col("asian").cast(BooleanType()))
pums = pums.withColumn("married", col("married").cast(BooleanType()))

sa = SampleAggregate(pums)
#print(sa.parts)

In [11]:
from itertools import product
cols = ["sex", "married", "income", "age", "educ"] # needs to be a superset of keys
keys = ["married","educ"]
groups = list(product([True, False], range(15)))

In [12]:
sa.sample(cols)

In [13]:
sa.aggregate(["income", "age"], keys, groups) # cols here doesn't need to include keys.  this bootstraps

In [14]:
def mean_income(data):
    cols = list(zip(*list(data)))
    income = cols[0]
    weights = cols[-1]
    weighted_sum = np.sum([i * w for i, a, w in data])
    return float(weighted_sum / np.sum(weights))

def count(data):
    cols = list(zip(*list(data)))
    weights = cols[-1]
    return int(np.sum(weights))

def mean_age(data):
    cols = list(zip(*list(data)))
    age = cols[1]
    weights = cols[-1]
    weighted_sum = np.sum([a * w for i, a, w in data])
    return float(weighted_sum / np.sum(weights))


In [15]:
sa.apply([mean_income, mean_age, count])


In [16]:
import math

def mean_estimator(data, eps, delta, lam, parts):    
    if lam is not None:
        data = [-lam if d < -lam else lam if d > lam else d for d in data]
    sd = (2 * lam * math.sqrt(2 * math.log(1.25 / delta)))/(parts*eps)
    np.random.seed()
    noise = np.random.normal(0, sd)
    theta = float(np.nanmean(data))
    return theta + noise
    
def median_estimator(data, eps, delta, lam, parts):
    # not private
    return float(np.median(data))

In [19]:
eps = 1.0
delta = 1E-9
sens = [100000, 65, 350000]

est_mean = sa.estimate(mean_estimator, eps, delta, sens)

est_med = sa.estimate(median_estimator, eps, delta, sens)


est_mean.toDF().show(50, False)
est_med.toDF().show(50, False)

+-----------+------------------------------------------------------------+
|group      |val                                                         |
+-----------+------------------------------------------------------------+
|[false, 14]|[50428.26957969916, 46.19695385335781, 26952.342319707775]  |
|[true, 9]  |[24560.545687125294, 47.59327614914312, 134551.79045907376] |
|[true, 7]  |[17070.579585586496, 42.95948588440542, 22355.693695357382] |
|[false, 1] |[10071.03036948753, 47.50932492901774, 22238.945711547734]  |
|[true, 14] |[61954.77029106083, 48.47397611664054, 43668.03412397737]   |
|[false, 10]|[21478.477544678844, 37.759712134220315, 45567.8144773766]  |
|[true, 4]  |[15132.513408887662, 47.78808816016781, 23865.848445702017] |
|[false, 4] |[12443.997639276222, 49.168307830482576, 20565.98223656074] |
|[true, 3]  |[13076.213206275539, 43.10325238019989, 33383.24618549618]  |
|[false, 13]|[43322.73776076855, 41.14059098996164, 75532.39663936813]   |
|[true, 10] |[29589.65226